In [ ]:
%ls

In [ ]:
%mkdir -p data/valid
%mkdir -p data/sample
%mkdir -p data/sample/test/unknown
%cd data/train

In [ ]:
from shutil import copyfile
def copy(files, path, destination_folder):
    for file in files:
        current_file = "./" + path + "/" + file
        destination_file = destination_folder + "/" + file
        print(current_file, "->", destination_file)
        print("cp -p " + current_file + " " + destination_file)
        copyfile(current_file, destination_file)    

In [ ]:
import os
import math
import numpy as np
from subprocess import call
import csv
from itertools import groupby

sample_rate = 0.10

In [ ]:
test_files = list(os.listdir("../test/unknown"))
sample_size = len(test_files) * sample_rate
test_files = np.random.choice(test_files, sample_size)
test_files_path = "../sample/test/unknown"
copy(test_files, "../test/unknown", test_files_path)

In [ ]:
img_list_file = open("../../driver_imgs_list.csv", "r")
img_list = list(csv.reader(img_list_file, delimiter=","))[1:]
drivers = groupby(img_list, lambda x: x[0])

In [ ]:
valid_destination_folder = "../sample/valid/"
train_destination_folder = "../sample/train/"

def copy_driver(driver, destination_path):
    driver_actions = groupby(driver[1], lambda x: x[1])
    driver_name = driver[0]
    for action in driver_actions:
        action_name = action[0]
        image_names = map(lambda x: x[2], action[1])
        destination_folder = destination_path + action_name
#         total_size = len(image_names)
#         sample_size = math.floor(total_size * sample_rate)
#         file_names = np.random.choice(image_names, sample_size)

        print("Driver", driver_name, action_name, len(image_names))

        print("File_names", image_names)
        call("mkdir -p " + destination_folder + "/" + action_name, shell=True)
        copy(image_names, action_name, destination_folder)

index = 0
for driver in drivers:
    if index < 10:
        copy_driver(driver, train_destination_folder)
    elif index > 10 and index < 13:
        copy_driver(driver, valid_destination_folder)
    index = index + 1

In [ ]:
%cd ../..

In [ ]:
%mkdir -p results
%ls

In [1]:
from model import Model as CNNModel
from keras.preprocessing import image

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
cnn = CNNModel()

In [3]:
batch_size = 64
path = "./data/sample/";

In [4]:
train_batches = cnn.get_batches(path + "train", batch_size=batch_size, gen=image.ImageDataGenerator(rotation_range=10))
valid_batches = cnn.get_batches(path + "valid", batch_size=batch_size, shuffle=False)
test_batches  = cnn.get_batches(path + "test", batch_size=batch_size, shuffle=False)

Found 10043 images belonging to 10 classes.
Found 1315 images belonging to 10 classes.
Found 7586 images belonging to 1 classes.


In [ ]:
cnn.finetune(train_batches)

In [ ]:
cnn.fit(train_batches, valid_batches, nb_epoch=1)

Epoch 1/1
 2304/10043 [=====>........................] - ETA: 713s - loss: 14.1540 - acc: 0.1050

In [ ]:
predictions = cnn.model.predict_generator(test_batches, len(test_batches.filenames), max_q_size=batch_size*2)

In [ ]:
for p in predictions:
    print(p) 